<a href="https://colab.research.google.com/github/SaketMunda/transfer-learning-with-tensorflow/blob/master/fine_tuning_transfer_learning_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with TensorFlow : Fine Tuning

In the previous section, we saw how we could leverage feature extraction transfer learning to get far better results on our Food vision project (using only 10% of original dataset) than building our own model from scratch.

Now, we're going to cover another type of transfer learning: fine-tuning.

In **fine-tuning transfer learning** the pre-trained weights from another model are unfrozen and tweaked during to better suit your own data.

*Feature extraction transfer learning vs. fine-tuning transfer learning. The main difference between the two is that in fine-tuning, more layers of the pre-trained model get unfrozen and tuned on custom data. This fine-tuning usually takes more data than feature extraction to be effective.*

> 💡 **This time we will use the helper functions to speed up our steps in learning**

Import the Helper functions

In [23]:
# Get helper_functions.py script from Github
!wget https://raw.githubusercontent.com/SaketMunda/ml-helpers/master/helper_functions.py

# Import helper functions we're going to use
from helper_functions import create_tensorboard_callback, unzip_data, walk_through_dir

--2022-12-16 05:19:08--  https://raw.githubusercontent.com/SaketMunda/ml-helpers/master/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1937 (1.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   1.89K  --.-KB/s    in 0s      

2022-12-16 05:19:09 (35.7 MB/s) - ‘helper_functions.py’ saved [1937/1937]



ImportError: ignored

## 10 Food Classes : Working with Less data

As in the previous notebook of Transfer Leanring : Feature Extraction, we got good results from less data (10%) of the training data using transfer learning from TensorFlow Hub.

In this notebook, we're going to continue to work with smaller subsets of the data, except this time we'll have a look at how we can use in-built pretrained models within the `tf.keras.applications` module as well as how to fine-tune them to our own custom dataset.

We'll also practice using a new but similar dataloader function to what we've used before, `image_dataset_from_directory()` which is a part of `tf.keras.preprocessing` module.

Finally we'll be practicing using the [Keras Functional API](https://keras.io/guides/functional_api/) for building deep learning models. The functional API is a more flexible way to create models than the `tf.keras.Sequential`

Let's start by downloading the data

In [2]:
# Get 10% of the data of the 10 classes
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data('10_food_classes_10_percent.zip')

--2022-12-16 04:20:42--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  30.8MB/s    in 6.1s    

2022-12-16 04:20:48 (26.4 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [3]:
# Walk through the directory and list number of files
walk_through_dir('10_food_classes_10_percent')

There are 2 directories and 0 files in '10_food_classes_10_percent'
There are 10 directories and 0 files in '10_food_classes_10_percent/train'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/hamburger'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/chicken_curry'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/steak'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/grilled_salmon'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/chicken_wings'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/pizza'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/fried_rice'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/ice_cream'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/sushi'
There are 0 directories and 75 files in '10_food_classes_10_percent/train/ramen'
There are

It's the same number of files and classes we used in previous notebook.

In [4]:
# Create the training and testing directory
train_dir = '10_food_classes_10_percent/train/'
test_dir = '10_food_classes_10_percent/test/'

Now we've got some image data, we need a way of loading it into a Tensorflow compatible format.

Previously, we've used the `ImageDataGenerator` class. And while this works well and is still very commonly used, this time we're going to use the `image_dataset_from_directory` function.

One of the main benefits of using `tf.keras.preprocessing.image_dataset_from_directory()` rather than `ImageDataGenerator` is that it creates a `tf.data.Dataset` object rather than a generator. The main advantage of this is the `tf.data.Dataset` API is much faster and efficient than the `ImageDataGenerator` API which is paramount for larger datasets.



In [10]:
import tensorflow as tf

IMG_SHAPE = (224, 224)
BATCH_SIZE = 32

train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                  image_size=IMG_SHAPE,
                                                  label_mode='categorical',
                                                  batch_size=BATCH_SIZE)

test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                 image_size=IMG_SHAPE,
                                                 label_mode='categorical',
                                                 batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


Wonderful ! Looks like our dataloaders have found the correct number of images for each Dataset.

For now, the main parameters we're concerned about in the `image_dataset_from_directory` function are:
- `directory`
- `image_size`
- `batch_size`

In [15]:
# if we check the datatype of the preprocessed dataset
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

It's Batch Dataset

In [17]:
# check the classes
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

Or if we wanted to see an example of batch of data, we could use the `take()` method

In [18]:
# See an example batch of data
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[2.18262909e+02 2.20262909e+02 2.18548630e+02]
   [2.29039383e+02 2.32110809e+02 2.36825104e+02]
   [2.29207581e+02 2.33408005e+02 2.46341202e+02]
   ...
   [7.49052277e+01 6.73337555e+01 5.61194954e+01]
   [7.19239349e+01 6.19239349e+01 5.19239349e+01]
   [6.37116776e+01 5.37116776e+01 4.37116776e+01]]

  [[2.28361130e+02 2.30361130e+02 2.28563614e+02]
   [2.34136154e+02 2.37746170e+02 2.40720993e+02]
   [2.33700409e+02 2.35557068e+02 2.48084351e+02]
   ...
   [7.30350189e+01 6.44635468e+01 5.52492867e+01]
   [7.34839478e+01 6.34839516e+01 5.44839516e+01]
   [6.31825066e+01 5.31825066e+01 4.41825066e+01]]

  [[2.40898758e+02 2.42898758e+02 2.39898758e+02]
   [2.32969070e+02 2.33210617e+02 2.35470337e+02]
   [2.28244095e+02 2.27255264e+02 2.36729263e+02]
   ...
   [7.18604126e+01 6.32889442e+01 5.56237869e+01]
   [7.14087067e+01 6.21832657e+01 5.47323723e+01]
   [6.52837906e+01 5.60583420e+01 4.86074524e+01]]

  ...

  [[9.35490723e+01 1.10549072e+02 1.30098145e+02]
   [9

Image arrays come out as tensors of pixel values where as the labels come out as one-hot encodings.

## Model 0. Building a transfer learning model using the Keras Functional API

Alight, our data is tensor-ified, let's build a model.

To do so, we're going to be using the `tf.keras.applications` module as it contains a series of already trained (on ImageNet) computer vision models as well as the Keras Functional API to construct our model.

We're going to go through the following steps:

1. Instantiate the pretrained model object by choosing a target model such as `EfficientNetB0` from `tf.keras.applications`, setting the `include_top` to `False` (we do this because we're going to create our own top, which are the output layers for the model).
2. Set the base model's `trainable` attribute to `False` to freeze all of the weights in the pre-trained model.
3. Define an input layer for our model, for example, what shape of data should our model expect ?
4. [Optional] Normalize the inputs to our model if it requires. Some of comp vis models such as `ResNetV250` require their inputs to be between 0 & 1.

In [ ]:
# 1. Create a base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model (so the pre-trained weights remain same)
base_model.trainable = False

# 3. Create the inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name='input_layer')

# 4. If using ResNet50V2, add this to speed up convergence, remove for EfficientNet
# x = tf.keras.layers.experimental.preprocessing.Rescaling(1/255.)(inputs)

# 5. Pass the inputs to our base model
x = base_model(inputs)
# check data shape after passing into base model
print(f'Shape after base_model: {x.shape}')

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computation)
x = tf.keras.layers.GlobalAveragePooling2D(name='global_average_pooling_layer')(x)
print(f'After Global Average Pooling 2D:{x.shape}')

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation='softmax', name='output_layer')(x)

# 8. Combine the inputs and outputs into our model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                optimizer='Adam',
                metrics=['accuracy'])

# 10. Fit the model
history_0 = model_0.fit(train_data_10_percent,
                        epochs=5,
                        steps_per_epochs=len(train_data_10_percent),
                        validation_data=test_data_10_percent,
                        validation_steps=len(test_data_10_percent),
                        callbacks=[create_tensorboard_callback('transfer_learning',
                                                               '10_percent_feature_extraction')])